In [475]:
'''
M1. Integradora


Authors:
Luis Alberto Alcántara Cabrales A01634185
Alexa Serrano Negrete A01654063
Renet de Jesús Pérez Gómez A01640555
Vicente Javier Viera Guízar A01639784

Date:
11/17/2022
'''

#Model
import agentpy as ap
import random


#Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython


In [476]:
parameters = {
    #Cantidad de carros
    'K': 20, 
    #Tamaño de ciudad
    'citySize': 60,
    #Duración simulación
    'steps': 300,
}

In [477]:
class CarAgent(ap.Agent):
    def setup(self): 
        '''
        0 = left 
        1 = right
        2 = top
        3 = botton
        '''       
        self.idType = random.randint(0, 3)
        # self.idType = random.choice([0,1])
        # self.idType = 2
        self.initialIdType = self.idType
        self.carChoseAWay = 0
        self.wayChosen = 0

class StreetAgent(ap.Agent):
    def setup(self):
       self.idType = 0

class TrafficLightAgent(ap.Agent):
    def setup(self):
        self.idType = 0
        self.timeCounterState = 0
        self.timeCounterWait = 0
        self.timeToStop = 10
        self.actualState = 0

    def determinateTime(self):
        if self.idType == 2 or self.idType == 3:
            self.timeCounterWait = self.timeToStop

    def updateState(self):
        self.actualState = 2 # Go
        if self.timeCounterWait > 0:
            #Time to wait
            self.timeCounterWait -= 1
            self.actualState = 4 # Stop
        else:
            #Time to work             
            if self.timeCounterState >= self.timeToStop - 3 and self.timeCounterState <= self.timeToStop - 1:
                self.actualState = 3 # Caution
            elif self.timeCounterState >= self.timeToStop:
                self.actualState = 4 # Stop
                self.timeCounterState = 0
                self.timeCounterWait = self.timeToStop
            self.timeCounterState += 1
        

    def currentState(self):
        return self.actualState
        

In [478]:
def initialPositionStreetsAndAssignType(self):
    # Assign type of street
    aux = 0
    typeOfStreet = 0
    for street in self.streets:
        if aux <= self.endEnviroment:
            street.idType = typeOfStreet
            aux += 1
        else:
            typeOfStreet += 1
            street.idType = typeOfStreet
            aux = 1

    # Get initial position
    position = []
    
    for i in range(self.endEnviroment):
        position.append((self.leftSideStreet, i))    
    position.append((self.leftSideStreet, self.endEnviroment))    

    for i in range(self.endEnviroment):
        position.append((self.rightSideStreet, i))
    position.append((self.rightSideStreet, self.endEnviroment))        
    
    for i in range(self.endEnviroment):
        position.append((i, self.rightSideStreet))
    position.append((self.endEnviroment, self.rightSideStreet))
    
    for i in range(self.endEnviroment):
        position.append((i, self.leftSideStreet))
    position.append((self.endEnviroment, self.leftSideStreet))
    
    return position

In [479]:
class MyModel(ap.Model):
    
    def setup(self):
        # Find special coordinates
        
        # Fundamental coordinates
        self.leftSideStreet = int(self.p.citySize / 2)
        self.rightSideStreet = self.leftSideStreet + 1
        self.endEnviroment = self.p.citySize - 1

        # Find the coordinates of center 
        self.centerCoordinates = [
            [self.rightSideStreet, self.leftSideStreet],
            [self.leftSideStreet, self.rightSideStreet],
            [self.leftSideStreet, self.leftSideStreet],
            [self.rightSideStreet, self.rightSideStreet],
        ]

        # Find crosswalk
        self.crosswalkCoordinates = [
            [self.rightSideStreet, self.leftSideStreet - 2],
            [self.leftSideStreet, self.rightSideStreet + 2],
            [self.leftSideStreet - 2, self.leftSideStreet],
            [self.rightSideStreet + 2, self.rightSideStreet],
        ]

        # Find initial positions
        self.initialPositionCoordinates = [
            [self.rightSideStreet, 0],
            [self.leftSideStreet, self.endEnviroment],
            [0, self.leftSideStreet],
            [self.endEnviroment, self.rightSideStreet]
        ]

        # Find final positions
        self.finalPositionCoordinates = [
            [self.rightSideStreet, self.endEnviroment],
            [self.leftSideStreet, 0],
            [self.endEnviroment, self.leftSideStreet],
            [0, self.rightSideStreet]
        ]
        
        # Store type of movement
        self.move = [
            [0, 1], [0, -1], [1, 0], [-1, 0],
            [-1, 1], [1, -1], [1, 1], [-1, -1]
        ]

        # We store this to query in the semaphore (This is the only decision that you can ignore the state of the semaphore)
        self.rightSides = [2, 3, 1, 0]

        self.leftSides = [3, 2, 0, 1]

        # Create agents
        self.cars = ap.AgentList(self, self.p.K, CarAgent)
        self.streets = ap.AgentList(self, int(self.p.citySize * 4), StreetAgent)
        self.trafficLights = ap.AgentList(self, 4, TrafficLightAgent)
        

        # Create enviroment
        self.area = ap.Grid(self, [self.p.citySize] * 2)


        # Add agents to the enviroment
        #Cars
        positionCars = []
        for car in self.cars:
            positionCars.append(self.initialPositionCoordinates[car.idType])
            
        self.area.add_agents(self.cars, positionCars)
        
        # Street
        positionStreet = initialPositionStreetsAndAssignType(self)
        # self.area.add_agents(self.streets, positionStreet)
        
        #trafficLights - fix the positions
        '''
            The position in the grid of the traffic light doesn't import 
            it's more about decoration. The position in the "array" 
            is what really really
        '''
        positionTrafficLight = [
            [self.crosswalkCoordinates[3][0] - 1, self.crosswalkCoordinates[3][1] + 2], # Traffic light for street 0
            [self.crosswalkCoordinates[2][0] + 1, self.crosswalkCoordinates[2][1] - 2], # Traffic light for street 1
            [self.crosswalkCoordinates[0][0] + 2, self.crosswalkCoordinates[0][1] + 1], # Traffic light for street 2
            [self.crosswalkCoordinates[1][0] - 2, self.crosswalkCoordinates[1][1] - 1], # Traffic light for street 3
        ]

        self.area.add_agents(self.trafficLights, positionTrafficLight)
        
        #Assign the type of traffic light and its time to works
        counterTrafficLight = 0
        for trafficLight in self.trafficLights:
            trafficLight.idType = counterTrafficLight
            trafficLight.determinateTime()
            counterTrafficLight += 1


        # Define type of agent
        self.cars.agent_type = 0
        self.streets.agent_type = 1

        for trafficLight in self.trafficLights:
            if trafficLight.idType == 0 or trafficLight.idType == 1:
                trafficLight.agent_type = 2 # That type of traffic light start in "Green"
            elif trafficLight.idType == 2 or trafficLight.idType == 3:
                trafficLight.agent_type = 4 # That type of traffic light start in "Red"      


            

    def step(self):

        # Update state of the traffic lights
        for trafficLight in self.trafficLights:
            trafficLight.updateState()
            trafficLight.agent_type = trafficLight.currentState()

        for car in self.cars:
            [crossY, crossX] = self.crosswalkCoordinates[car.initialIdType]
            [yCar, xCar] = self.area.positions[car]
            trafficLightState = self.trafficLights[car.initialIdType].currentState()
            stop = 0
            carInFront = 0
            diagonalMovement = 0

            # The current car reach the crosswalk and choose a way        
            if (self.crosswalkCoordinates[car.initialIdType][0] == yCar
                and self.crosswalkCoordinates[car.initialIdType][1] == xCar
                and car.carChoseAWay == 0):
                    if car.initialIdType == 0:
                        car.wayChosen = random.choice([0, 2, 3])
                        # car.wayChosen = 3
                        
                    if car.initialIdType == 1:
                        car.wayChosen = random.choice([1, 2, 3])
                        # car.wayChosen = 2
                        
                    if car.initialIdType == 2:
                        car.wayChosen = random.choice([0, 1, 2])
                        # car.wayChosen = 0
                        
                    if car.initialIdType == 3:
                        car.wayChosen = random.choice([0, 1, 3])
                        # car.wayChosen = 1
                        
                    car.carChoseAWay = 1 # This attribute helps us prevent the car from choose a way again

            # Verify if it is necessary to stop
            if(yCar == crossY and crossX == xCar and trafficLightState != 2 
            and self.rightSides[car.initialIdType] != car.wayChosen):
                # print("The car: ", car.id, ", chose: ", car.wayChosen, " - right side: ", self.rightSides[car.initialIdType], " - initial type: ", car.initialIdType)
                # print("Alto")
                stop = 1

            # Verify if the car reach the center
            [yCoor, xCoor] = self.centerCoordinates[car.initialIdType]
            if yCoor == yCar and xCoor == xCar and self.leftSides[car.initialIdType] != car.wayChosen:
                car.idType = car.wayChosen # Change direction

            if stop == 0:
                # print("car id: ", car.id, " - chose: ", car.wayChosen)
                # Verify if there's a neighbor type car    
                for neighbor in self.area.neighbors(car):
                    # Verify if there's a neighbor car
                    if neighbor.agent_type == 0:   
                        [yNeighbor, xNeighbor] = self.area.positions[neighbor]
                        posY = yNeighbor - yCar
                        posX = xNeighbor - xCar 
                        [movY, movX] = self.move[car.idType] # Get value movement
                        
                        # Check forward movement
                        if (posY == movY and movX == posX):
                            carInFront = 1

                        # Check left movement
                        if yCoor == yCar and xCoor == xCar and self.leftSides[car.initialIdType] == car.wayChosen:
                            print("Is a left check")
                            [movY, movX] = self.move[car.idType + 4] # Get the value movement
                            if (posY == movY and movX == posX):
                                print("You can move in diagonal")
                                diagonalMovement = 1
                                self.area.move_by(car, self.move[car.idType + 4])
                                car.idType = car.wayChosen # Change direction
                            else:
                                self.area.move_by(car, self.move[car.idType])
                                car.idType = car.wayChosen # Change direction
                            carInFront = 1
                            break

                # Move the car
                if carInFront == 0:    
                    self.area.move_by(car, self.move[car.idType])
                    # Moves the car
                    # if diagonalMovement == 1 and yCoor == yCar and xCoor == xCar:
                    #     self.area.move_by(car, self.move[car.idType + 4])
                    # else:
                    #     self.area.move_by(car, self.move[car.idType])


            # Verify if the car reached the destination coordinates 
            [currentY, currentX] = self.area.positions[car]
            [destinationY, destinationX] = self.finalPositionCoordinates[car.idType]
            if currentY == destinationY and destinationX == currentX:
                car.agent_type = 1

        
        # print("--------------------")
        # for car in self.cars: 
        #     print("The car: ", car.id, ", chose: ", car.wayChosen)

        # If there's not an agent car of type 0 then we finish the simulation
        finishSimulation = 1
        for car in self.cars: 
            if car.agent_type == 0:
                finishSimulation = 0
                break
            
        if finishSimulation == 1:
            self.stop()


In [480]:
# Create single-run animation with custom colors

def animation_plot(model, ax):
    attr_grid = model.area.attr_grid('agent_type')
    color_dict = {0:'#c1c1c1', 1: '#007', 2: '#3ECC06', 3: '#F7F30E', 4: '#FF5733', None:'#FFF'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Traffic Simulation\n"
                 f"Time-step: {model.t}",
                 color="white") 

fig, ax = plt.subplots()
model = MyModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=30))

Is a left check
You can move in diagonal
Is a left check
Is a left check
Is a left check
Is a left check
Is a left check
Is a left check
Is a left check
